# Import thư viện

In [1]:
pip install evaluate

In [2]:
pip install rouge_score

In [3]:
pip install pyvi spacy

In [4]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import spacy
from pyvi import ViTokenizer
from collections import Counter

In [6]:
from torch import nn
import torch.optim as optim


In [7]:
import os

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import sys
sys.path.append('/content/drive/MyDrive/DS201')

In [10]:
from data_utils.build_vocab import Vocabulary,tokenize_en,tokenize_vi
from data_utils.make_pytorch_dataset import TranslationDataset, MyCollate


In [11]:
from models.luong_encode_decode_model import Encoder, LuongDecoder, Seq2Seq
from training import train_model, eval_model


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
train_ds = pd.read_json(r'/content/drive/MyDrive/DS201/small-PhoMT/small-train.json')
dev_ds = pd.read_json(r'//content/drive/MyDrive/DS201/small-PhoMT/small-dev.json')
test_ds = pd.read_json(r'/content/drive/MyDrive/DS201/small-PhoMT/small-test.json')

In [14]:
train_ds

,english,vietnamese
0,It begins with a countdown .,Câu chuyện bắt đầu với buổi lễ đếm ngược .
1,"On August 14th , 1947 , a woman in Bombay goes...","Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở..."
2,"Across India , people hold their breath for th...","Cùng lúc , trên khắp đất Ấn , người ta nín thở..."
3,"And at the stroke of midnight , a squirming in...","Khi đồng hồ điểm thời khắc nửa đêm , một đứa t..."
4,"These events form the foundation of "" Midnight...","Những sự kiện này là nền móng tạo nên "" Những ..."
...,...,...
19995,And the man was incredibly curious .,Và người đàn ông này cực kỳ tò mò .
19996,And he wanted to understand what it was and wh...,Và ông muốn hiểu nó là gì và tại sao thế rằng ...
19997,"And one day , we were walking .","Một ngày nọ , chúng tôi đang đi bộ ."
19998,"We were in France , in Les Houches .","Chúng tôi ở Pháp , tại Les Houches ."


In [15]:
dev_ds

,english,vietnamese
0,It begins with a countdown .,Câu chuyện bắt đầu với buổi lễ đếm ngược .
1,"On August 14th , 1947 , a woman in Bombay goes...","Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở..."
2,"Across India , people hold their breath for th...","Cùng lúc , trên khắp đất Ấn , người ta nín thở..."
3,"And at the stroke of midnight , a squirming in...","Khi đồng hồ điểm thời khắc nửa đêm , một đứa t..."
4,"These events form the foundation of "" Midnight...","Những sự kiện này là nền móng tạo nên "" Những ..."
...,...,...
19995,And the man was incredibly curious .,Và người đàn ông này cực kỳ tò mò .
19996,And he wanted to understand what it was and wh...,Và ông muốn hiểu nó là gì và tại sao thế rằng ...
19997,"And one day , we were walking .","Một ngày nọ , chúng tôi đang đi bộ ."
19998,"We were in France , in Les Houches .","Chúng tôi ở Pháp , tại Les Houches ."


In [16]:
test_ds

,english,vietnamese
0,It begins with a countdown .,Câu chuyện bắt đầu với buổi lễ đếm ngược .
1,"On August 14th , 1947 , a woman in Bombay goes...","Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở..."
2,"Across India , people hold their breath for th...","Cùng lúc , trên khắp đất Ấn , người ta nín thở..."
3,"And at the stroke of midnight , a squirming in...","Khi đồng hồ điểm thời khắc nửa đêm , một đứa t..."
4,"These events form the foundation of "" Midnight...","Những sự kiện này là nền móng tạo nên "" Những ..."
...,...,...
19995,And the man was incredibly curious .,Và người đàn ông này cực kỳ tò mò .
19996,And he wanted to understand what it was and wh...,Và ông muốn hiểu nó là gì và tại sao thế rằng ...
19997,"And one day , we were walking .","Một ngày nọ , chúng tôi đang đi bộ ."
19998,"We were in France , in Les Houches .","Chúng tôi ở Pháp , tại Les Houches ."


# Xây dựng vocab và pytorch dataset

In [17]:
BATCH_SIZE = 32


In [19]:
# Đường dẫn file muốn lưu
file_vocab_en = "/content/drive/MyDrive/DS201/vocab_en_new.pth"
file_vocab_vi = "/content/drive/MyDrive/DS201/vocab_vi_new.pth"

# Khởi tạo object rỗng
vocab_en = Vocabulary(freq_threshold=4)
vocab_vi = Vocabulary(freq_threshold=4)

# --- XỬ LÝ CHO TIẾNG ANH ---
if os.path.exists(file_vocab_en):
    # TRƯỜNG HỢP 1: Đã có file -> Load lên
    print("Đang tải từ điển tiếng Anh từ file...")
    vocab_en.load_vocab(file_vocab_en)
else:
    # TRƯỜNG HỢP 2: Chưa có file -> Build rồi Save
    print("Đang xây dựng từ điển tiếng Anh (lần đầu)...")
    vocab_en.build_vocabulary(train_ds['english'].tolist(), tokenize_en)
    vocab_en.save_vocab(file_vocab_en)

# --- XỬ LÝ CHO TIẾNG VIỆT ---
if os.path.exists(file_vocab_vi):
    print("Đang tải từ điển tiếng Việt từ file...")
    vocab_vi.load_vocab(file_vocab_vi)
else:
    print("Đang xây dựng từ điển tiếng Việt (lần đầu)...")
    vocab_vi.build_vocabulary(train_ds['vietnamese'].tolist(), tokenize_vi)
    vocab_vi.save_vocab(file_vocab_vi)

print("Hoàn tất chuẩn bị dữ liệu!")

print(f"Kích thước từ điển Anh: {len(vocab_en)}")
print(f"Kích thước từ điển Việt: {len(vocab_vi)}")

dataset_train = TranslationDataset(train_ds, vocab_en, vocab_vi)
dataset_dev = TranslationDataset(dev_ds, vocab_en, vocab_vi)
dataset_test = TranslationDataset(test_ds, vocab_en, vocab_vi)
# Padding index là 0 (đã định nghĩa trong Vocabulary class)
pad_idx = vocab_en.stoi["<pad>"]

# Tạo DataLoader
train_loader = DataLoader(
    dataset=dataset_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=MyCollate(pad_idx=pad_idx)
)

dev_loader = DataLoader(
    dataset=dataset_dev,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=MyCollate(pad_idx=pad_idx)
)

test_loader = DataLoader(
    dataset=dataset_test,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=MyCollate(pad_idx=pad_idx)
)





Đang tải từ điển tiếng Anh từ file...
Đã tải từ điển từ: /content/drive/MyDrive/DS201/vocab_en_new.pth (Size: 5985)
Đang tải từ điển tiếng Việt từ file...
Đã tải từ điển từ: /content/drive/MyDrive/DS201/vocab_vi_new.pth (Size: 4913)
Hoàn tất chuẩn bị dữ liệu!
Kích thước từ điển Anh: 5985
Kích thước từ điển Việt: 4913


# Định nghĩa các siêu tham số

In [20]:
MAX_LENGTH = 50
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_vi)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HIDDEN_DIM = 256   # Yêu cầu
N_LAYERS = 3       # Yêu cầu
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_EPOCHS = 10
CLIP = 1

# Khởi tạo mô hình

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- KHỞI TẠO MÔ HÌNH ---
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT)
dec = LuongDecoder(OUTPUT_DIM, DEC_EMB_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

# --- KHỞI TẠO TỐI ƯU HÓA VÀ HÀM LOSS ---
optimizer = optim.Adam(model.parameters()) # Yêu cầu đề bài: Adam
criterion = nn.CrossEntropyLoss(ignore_index=0) # Giả sử index 0 là padding

# Giả sử chúng ta có 2 dict mapping từ vựng
vocab_trg_stoi = vocab_vi.stoi
vocab_trg_itos = vocab_vi.itos


# Train and Evaluate

In [22]:

# --- VÒNG LẶP HUẤN LUYỆN ---
print("Bắt đầu huấn luyện...")
for epoch in range(N_EPOCHS):
    train_loss = train_model(model, train_loader, optimizer, criterion, CLIP, device)
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')
    rouge_l_score = eval_model(model, dev_loader, vocab_trg_stoi, vocab_trg_itos,device)
    print(f'Epoch: {epoch+1:02} | ROUGE-L Score: {rouge_l_score['rougeL']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge1']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge2']:.2f}')


Bắt đầu huấn luyện...
Epoch: 01 | Train Loss: 6.019
Epoch: 01 | ROUGE-L Score: 0.09 | ROUGE-1-Score: 0.10 | ROUGE-1-Score: 0.00
Epoch: 02 | Train Loss: 5.785
Epoch: 02 | ROUGE-L Score: 0.09 | ROUGE-1-Score: 0.10 | ROUGE-1-Score: 0.01
Epoch: 03 | Train Loss: 5.603
Epoch: 03 | ROUGE-L Score: 0.21 | ROUGE-1-Score: 0.25 | ROUGE-1-Score: 0.07
Epoch: 04 | Train Loss: 5.405
Epoch: 04 | ROUGE-L Score: 0.25 | ROUGE-1-Score: 0.30 | ROUGE-1-Score: 0.10
Epoch: 05 | Train Loss: 5.242
Epoch: 05 | ROUGE-L Score: 0.30 | ROUGE-1-Score: 0.37 | ROUGE-1-Score: 0.13
Epoch: 06 | Train Loss: 5.112
Epoch: 06 | ROUGE-L Score: 0.32 | ROUGE-1-Score: 0.40 | ROUGE-1-Score: 0.14
Epoch: 07 | Train Loss: 4.988
Epoch: 07 | ROUGE-L Score: 0.33 | ROUGE-1-Score: 0.41 | ROUGE-1-Score: 0.16
Epoch: 08 | Train Loss: 4.884
Epoch: 08 | ROUGE-L Score: 0.34 | ROUGE-1-Score: 0.42 | ROUGE-1-Score: 0.17
Epoch: 09 | Train Loss: 4.781
Epoch: 09 | ROUGE-L Score: 0.35 | ROUGE-1-Score: 0.44 | ROUGE-1-Score: 0.18
Epoch: 10 | Train Loss: 

# Test

In [23]:
rouge_l_score = eval_model(model, test_loader, vocab_trg_stoi, vocab_trg_itos,device)
print(f'Test : ROUGE-L Score: {rouge_l_score['rougeL']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge1']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge2']:.2f}')

Test : ROUGE-L Score: 0.36 | ROUGE-1-Score: 0.45 | ROUGE-1-Score: 0.19
